In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from urllib.request import urlopen
import bs4
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import os
import datetime
import time
import re
import dataset

In [2]:
url_fmt = "http://db.netkeiba.com"

In [30]:
driver = webdriver.PhantomJS(executable_path="/home/tsumita/webdriver/phantomjs")
# 要素が見つかるまで10秒待つ設定
driver.implicitly_wait(20)

driver.get(url_fmt)
print(driver.title)


/home/tsumita/anaconda3/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


netkeiba.com - 競馬データベース


In [31]:
trans_bir = str.maketrans({'年':'/','月':'/','日':''})
# trans_emp = str.maketrans({' ':'', ',':'', '万円':''})
simple_colmns = ['status', 'sex', 'color']

def extraction_review(review_list):
    ## 適性レビュー
    review_data = {}
    review_data['course_aptitude'] = int(review_list[0].find_all('img')[1].get('width'))
    review_data['direct_aptitude'] = int(review_list[1].find_all('img')[1].get('width'))
    review_data['race_type'] = int(review_list[2].find_all('img')[1].get('width'))
    review_data['growth'] = int(review_list[3].find_all('img')[1].get('width'))
    review_data['omobaba'] = int(review_list[4].find_all('img')[1].get('width'))
    return review_data
    
def overview_text(tag_info):
    if tag_info is None:
        return None
    else:
        return tag_info.text
    
def extraction_overview(sp):
    ## 馬概要
    overview_data = {}
    
    sim_info = overview_text(soup.find("p", class_="txt_01"))
    if sim_info is not None:
        sim_info_list = sim_info.split('\u3000')
        for s_col, s_info in zip(simple_colmns, sim_info_list):
            overview_data[s_col] = s_info
    
    overview_data['birthday'] = overview_text(sp.find('th', string='生年月日').parent.find('td')).translate(trans_bir)
    overview_data['trainer'] = overview_text(sp.find('th', string='調教師').parent.find('a'))
    overview_data['owner'] = overview_text(sp.find('th', string='馬主').parent.find('a'))
    overview_data['breeder'] = overview_text(sp.find('th', string='生産者').parent.find('a'))
    overview_data['breed_center'] = overview_text(sp.find('th', string='産地').parent.find('a'))
    overview_data['producefrom'] = overview_text(sp.find('th', string='産地').parent.find('td'))
    seri = overview_text(sp.find('th', string='セリ取引価格').parent.find('td'))
    if seri is None:
        overview_data['seri'] = None
    else:
        seri = seri.replace('\n', '')
        overview_data['seri'] = None if seri=='-' else seri
    prize = overview_text(sp.find('th', string='獲得賞金').parent.find('td'))
    if prize is None:
        overview_data['total_prize_chuo'] = None
        overview_data['total_prize_chiho'] = None
    else:
        prize = prize.replace('\n', '')
        overview_data['total_prize_chuo'] = float(prize.split('(中央)')[0].split('/')[-1].replace(' ','').replace('万円','').replace('億','').replace(',','')) if '(中央)' in prize else None
        overview_data['total_prize_chiho'] = float(prize.split('(地方)')[0].split('/')[-1].replace(' ','').replace('万円','').replace('億','').replace(',','')) if '(地方)' in prize else None
    record = overview_text(sp.find('th', string='通算成績').parent.find('td'))
    if record is None:
        overview_data['total_win'] = None
        overview_data['total_race'] = None
        overveew_data['total_result'] = None
    else:
        record = re.split('[戦勝\[\]]', record.replace(' ', ''))
        overview_data['total_race'] = int(record[0])
        overview_data['total_win'] = int(record[1])
        overview_data['total_result'] = record[3]
    return overview_data

def pedigree_info(sp):
    ## 血統情報
    pedigree_data = {}
    parents = sp.find_all("td", rowspan="2")

    fa = parents[0]
    grand_pa = fa.parent.parent.find_all('td')
    pedigree_data['father'] = grand_pa[0].find('a').text
    pedigree_data['pat_grandfa'] = grand_pa[1].find('a').text
    pedigree_data['pat_grandmo'] = fa.parent.find_next_siblings("tr")[0].find('a').text

    mo = parents[1]
    grand_mo = mo.parent.parent.find_all('td')
    pedigree_data['mother'] = grand_mo[0].find('a').text
    pedigree_data['mat_grandfa'] = grand_mo[1].find('a').text
    pedigree_data['mat_grandmo'] = mo.parent.find_next_siblings("tr")[0].find('a').text
    
    return pedigree_data


def extraction_horse_info(soup, tr):
    data = {}
    review_data = extraction_review(tr[0:5])
    data.update(review_data)
#     print("+"*30)
    overview_data = extraction_overview(soup)
    data.update(overview_data)
#     print("+"*30)
    pedigree_data = pedigree_info(soup)
    data.update(pedigree_data)
    return data



def create_horsedatadict(soup):
#     print("+"*30)
    tr = soup.find_all("tr")
    horse_data = extraction_horse_info(soup, tr)
    return horse_data

In [49]:
def get_db_settings():
    db_settings = {
        'host':'****',
        'user':'root',
        'password':'****',
        'database':'keiba',
        'charset':'utf8',
        'port':3306
    }
    return db_settings

def get_dataset_connection(db_settings):
    return dataset.connect('mysql://{user}:{password}@{host}:{port}/{database}?charset={charset}'.format(**db_settings))

def get_table(db, table):
    return db[table]

In [50]:
db = get_dataset_connection(get_db_settings())
horse_info_table = get_table(db, "horse_info")
insert_table = get_table(db, "horse_info_")

In [ ]:
# records = horse_info_table.find(_limit=10)
records = horse_info_table.find()
horse_info_list = []

# insert中断された時用
last_insert_info = insert_table.find_one(id = len(insert_table))
last_h = last_insert_info['h_name']
# last_h = "ジェイケイオペラ"
scraping_flag = False
# records = ['アズミノニフクカゼ']
for row, record in enumerate(records):
    horse_name = record['h_name']
    if horse_name != last_h and scraping_flag != True:
        continue
    elif scraping_flag != True:
        scraping_flag = True
        continue
    print(horse_name)

    driver.find_element_by_class_name("Txt_Form").send_keys(horse_name)
    driver.find_element_by_class_name("Submit_Btn").submit()

    wait = WebDriverWait(driver, 20)
    # タイトルタグに任意のテキストが含まれたタイトルが表示されるまで20秒待つ
    wait.until(EC.title_contains("馬"))
    driver.implicitly_wait(10)
    
    print(driver.title)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    if "競走馬検索結果" in driver.title:
        horse_page_info = soup.find_all("td", class_="xml txt_l")
        horse_page_info = horse_page_info + soup.find_all("td", class_="bml txt_l")
        print(horse_page_info)
        horse_page_info = [td.find("a", title=horse_name) for td in horse_page_info if td.find("a", title=horse_name) != None]
#         horse_page_info = soup.find_all("a", title=horse_name)
        print(horse_page_info)
        print("-"*50)
        for horse_info in horse_page_info:
            url = url_fmt + horse_info.get("href")
            html = urlopen(url)
            soup = BeautifulSoup(html, "html.parser")

            print(soup.find("p", class_="txt_01").text)

            horse_data = create_horsedatadict(soup)
            horse_data['h_name'] = horse_name
            horse_info_list.append(horse_data)
            print(horse_data)
            time.sleep(2)
            print("+"*50)

    else:
        try:
            print(soup.find("p", class_="txt_01").text)
            horse_data = create_horsedatadict(soup)
            horse_data['h_name'] = horse_name
            horse_info_list.append(horse_data)
            print(horse_data)
            time.sleep(2)
            print("+"*50)
        except:
            
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
    
# insert_table.insert_many(horse_info_list, chunk_size=5000)

ジェイケイカフェ
馬名[ジェイケイカフェ]の競走馬検索結果｜競馬データベース - netkeiba.com
[<td class="bml txt_l" nowrap="nowrap"><a href="/horse/2008100845/" title="ジェイケイカフェ">ジェイケイカフェ</a></td>, <td class="bml txt_l" nowrap="nowrap"><a href="/horse/2014105007/" title="ジェイケイカフェの2014">ジェイケイカフェの2014</a></td>, <td class="bml txt_l" nowrap="nowrap"><a href="/horse/2015106121/" title="ジェイケイカフェの2015">ジェイケイカフェの2015</a></td>]
[<a href="/horse/2008100845/" title="ジェイケイカフェ">ジェイケイカフェ</a>]
--------------------------------------------------
現役　牝10歳　青鹿毛
{'course_aptitude': 96, 'direct_aptitude': 77, 'race_type': 69, 'growth': 77, 'omobaba': 77, 'status': '現役', 'sex': '牝10歳', 'color': '青鹿毛', 'birthday': '2008/3/4', 'trainer': '坂本勝美', 'owner': '小谷野次郎', 'breeder': '藤川ファーム', 'breed_center': None, 'producefrom': '新ひだか町', 'seri': None, 'total_prize_chuo': None, 'total_prize_chiho': None, 'total_race': 3, 'total_win': 0, 'total_result': '0-0-0-3', 'father': 'マンハッタンカフェ', 'pat_grandfa': 'サンデーサイレンス', 'pat_grandmo': 'サトルチェンジ', 'mother': 'マンハッタンカフェ

馬名[ジェイミー]の競走馬検索結果｜競馬データベース - netkeiba.com
[<td class="xml txt_l" nowrap="nowrap"><a href="/horse/2007106161/" title="ジェイミー">ジェイミー</a></td>, <td class="bml txt_l" nowrap="nowrap"><a href="/horse/2013100666/" title="ジェイミーの2013">ジェイミーの2013</a></td>]
[<a href="/horse/2007106161/" title="ジェイミー">ジェイミー</a>]
--------------------------------------------------
抹消　牝　栗毛
{'course_aptitude': 58, 'direct_aptitude': 87, 'race_type': 116, 'growth': 87, 'omobaba': 87, 'status': '抹消', 'sex': '牝', 'color': '栗毛', 'birthday': '2007/3/2', 'trainer': '鮫島一歩', 'owner': 'グランプリ', 'breeder': '目黒牧場', 'breed_center': None, 'producefrom': '日高町', 'seri': None, 'total_prize_chuo': None, 'total_prize_chiho': None, 'total_race': 2, 'total_win': 0, 'total_result': '0-0-0-2', 'father': 'ネオユニヴァース', 'pat_grandfa': 'サンデーサイレンス', 'pat_grandmo': 'ポインテッドパス', 'mother': 'ネオユニヴァース', 'mat_grandfa': 'サンデーサイレンス', 'mat_grandmo': 'コンアモール', 'h_name': 'ジェイミー'}
++++++++++++++++++++++++++++++++++++++++++++++++++
ジェイラッカー
ジェイラッカー | 競走馬データ - ne

　牡　黒鹿毛
{'course_aptitude': 58, 'direct_aptitude': 58, 'race_type': 58, 'growth': 58, 'omobaba': 58, 'status': '', 'sex': '牡', 'color': '黒鹿毛', 'birthday': '2012/6/13', 'trainer': '保利良次', 'owner': '薪浦亨', 'breeder': '吉田三郎', 'breed_center': None, 'producefrom': '安平町', 'seri': None, 'total_prize_chuo': 630.0, 'total_prize_chiho': 399.0, 'total_race': 23, 'total_win': 5, 'total_result': '5-1-0-17', 'father': 'ジャングルポケット', 'pat_grandfa': 'トニービン', 'pat_grandmo': 'ダンスチャーマー', 'mother': 'ジャングルポケット', 'mat_grandfa': 'トニービン', 'mat_grandmo': 'オギティファニー', 'h_name': 'ジェット'}
++++++++++++++++++++++++++++++++++++++++++++++++++
ジェットカフェ
ジェットカフェ | 競走馬データ - netkeiba.com
　牡　青鹿毛
{'course_aptitude': 58, 'direct_aptitude': 58, 'race_type': 87, 'growth': 58, 'omobaba': 58, 'status': '', 'sex': '牡', 'color': '青鹿毛', 'birthday': '2007/4/15', 'trainer': '寺嶋正勝', 'owner': '組）ジェントルマンホ', 'breeder': '冨岡牧場', 'breed_center': None, 'producefrom': '浦河町', 'seri': None, 'total_prize_chuo': None, 'total_prize_chiho': 14.0, 'total_r

++++++++++++++++++++++++++++++++++++++++++++++++++
ジェットソング
ジェットソング | 競走馬データ - netkeiba.com
　牡　黒鹿毛
{'course_aptitude': 116, 'direct_aptitude': 58, 'race_type': 116, 'growth': 58, 'omobaba': 116, 'status': '', 'sex': '牡', 'color': '黒鹿毛', 'birthday': '2010/4/2', 'trainer': '塚田隆男', 'owner': '橋元勇氣', 'breeder': '追分ファーム', 'breed_center': None, 'producefrom': '安平町', 'seri': None, 'total_prize_chuo': 210.0, 'total_prize_chiho': 277.0, 'total_race': 91, 'total_win': 12, 'total_result': '12-6-5-68', 'father': 'マンハッタンカフェ', 'pat_grandfa': 'サンデーサイレンス', 'pat_grandmo': 'サトルチェンジ', 'mother': 'マンハッタンカフェ', 'mat_grandfa': 'サンデーサイレンス', 'mat_grandmo': 'Arutua', 'h_name': 'ジェットソング'}
++++++++++++++++++++++++++++++++++++++++++++++++++
ジェットタイガー
ジェットタイガー | 競走馬データ - netkeiba.com
　牡　鹿毛
{'course_aptitude': 58, 'direct_aptitude': 58, 'race_type': 58, 'growth': 58, 'omobaba': 58, 'status': '', 'sex': '牡', 'color': '鹿毛', 'birthday': '2009/3/20', 'trainer': '新井清重', 'owner': '木下孝丸', 'breeder': '前田宗将', 'breed_center': Non

++++++++++++++++++++++++++++++++++++++++++++++++++
ジェットローズ
ジェットローズ | 競走馬データ - netkeiba.com
抹消　牝　鹿毛
{'course_aptitude': 104, 'direct_aptitude': 104, 'race_type': 96, 'growth': 69, 'omobaba': 74, 'status': '抹消', 'sex': '牝', 'color': '鹿毛', 'birthday': '2006/5/19', 'trainer': '清水英克', 'owner': '本間茂', 'breeder': '中脇一幸', 'breed_center': None, 'producefrom': '様似町', 'seri': None, 'total_prize_chuo': 700.0, 'total_prize_chiho': None, 'total_race': 6, 'total_win': 1, 'total_result': '1-0-0-5', 'father': 'マイネルラヴ', 'pat_grandfa': 'Seeking the Gold', 'pat_grandmo': 'Heart of Joy', 'mother': 'マイネルラヴ', 'mat_grandfa': 'Seeking the Gold', 'mat_grandmo': 'ノーブルアクション', 'h_name': 'ジェットローズ'}
++++++++++++++++++++++++++++++++++++++++++++++++++
ジェットヴォイス
ジェットヴォイス | 競走馬データ - netkeiba.com
抹消　牡　鹿毛
{'course_aptitude': 72, 'direct_aptitude': 72, 'race_type': 92, 'growth': 69, 'omobaba': 58, 'status': '抹消', 'sex': '牡', 'color': '鹿毛', 'birthday': '2007/5/14', 'trainer': '石毛善彦', 'owner': '本間茂', 'breeder': '新井昭二', 'breed

...

++++++++++++++++++++++++++++++++++++++++++++++++++
ゼネラルリッシュ
ゼネラルリッシュ | 競走馬データ - netkeiba.com
　牡　鹿毛
{'course_aptitude': 58, 'direct_aptitude': 58, 'race_type': 58, 'growth': 58, 'omobaba': 58, 'status': '', 'sex': '牡', 'color': '鹿毛', 'birthday': '2015/4/13', 'trainer': '新山広道', 'owner': '榊原勝好', 'breeder': '飯島功典', 'breed_center': None, 'producefrom': '新冠町', 'seri': None, 'total_prize_chuo': None, 'total_prize_chiho': 35.0, 'total_race': 10, 'total_win': 1, 'total_result': '1-0-0-9', 'father': 'タイムパラドックス', 'pat_grandfa': 'ブライアンズタイム', 'pat_grandmo': 'ジョリーザザ', 'mother': 'タイムパラドックス', 'mat_grandfa': 'ブライアンズタイム', 'mat_grandmo': 'タイホウアイ', 'h_name': 'ゼネラルリッシュ'}
++++++++++++++++++++++++++++++++++++++++++++++++++
ゼネラルレジーナ
ゼネラルレジーナ | 競走馬データ - netkeiba.com
抹消　牝　黒鹿毛
{'course_aptitude': 58, 'direct_aptitude': 58, 'race_type': 58, 'growth': 58, 'omobaba': 58, 'status': '抹消', 'sex': '牝', 'color': '黒鹿毛', 'birthday': '2014/3/22', 'trainer': '小桧山悟', 'owner': 'ジェネラルレーシング', 'breeder': '川上牧場', 'breed_center': 

++++++++++++++++++++++++++++++++++++++++++++++++++
ゼルビーノ
ゼルビーノ | 競走馬データ - netkeiba.com
　牡　栗毛
{'course_aptitude': 87, 'direct_aptitude': 58, 'race_type': 1, 'growth': 87, 'omobaba': 58, 'status': '', 'sex': '牡', 'color': '栗毛', 'birthday': '2014/4/29', 'trainer': '渡部則夫', 'owner': '岩渕瑞生', 'breeder': 'ノーザンファーム', 'breed_center': None, 'producefrom': '安平町', 'seri': None, 'total_prize_chuo': 540.0, 'total_prize_chiho': 202.0, 'total_race': 26, 'total_win': 1, 'total_result': '1-2-2-21', 'father': 'ゴールドアリュール', 'pat_grandfa': 'サンデーサイレンス', 'pat_grandmo': 'ニキーヤ', 'mother': 'ゴールドアリュール', 'mat_grandfa': 'サンデーサイレンス', 'mat_grandmo': 'Simply Gorgeous', 'h_name': 'ゼルビーノ'}
++++++++++++++++++++++++++++++++++++++++++++++++++
ゼロカラノキセキ
ゼロカラノキセキ | 競走馬データ - netkeiba.com
抹消　牝　鹿毛
{'course_aptitude': 29, 'direct_aptitude': 101, 'race_type': 87, 'growth': 1, 'omobaba': 58, 'status': '抹消', 'sex': '牝', 'color': '鹿毛', 'birthday': '2013/4/27', 'trainer': '尾関知人', 'owner': '広尾レース', 'breeder': '木村秀則', 'breed_center': Non

++++++++++++++++++++++++++++++++++++++++++++++++++
ゼロワンシンジロー
ゼロワンシンジロー | 競走馬データ - netkeiba.com
抹消　牡　鹿毛
{'course_aptitude': 87, 'direct_aptitude': 101, 'race_type': 58, 'growth': 58, 'omobaba': 58, 'status': '抹消', 'sex': '牡', 'color': '鹿毛', 'birthday': '2009/2/17', 'trainer': '森田直行', 'owner': '国田正忠', 'breeder': '長浜牧場', 'breed_center': None, 'producefrom': '新冠町', 'seri': '525万円 (2010年 セレクトセール)', 'total_prize_chuo': 1845.0, 'total_prize_chiho': 79.0, 'total_race': 49, 'total_win': 8, 'total_result': '8-2-4-35', 'father': 'デュランダル', 'pat_grandfa': 'サンデーサイレンス', 'pat_grandmo': 'サワヤカプリンセス', 'mother': 'デュランダル', 'mat_grandfa': 'サンデーサイレンス', 'mat_grandmo': 'マキーナ', 'h_name': 'ゼロワンシンジロー'}
++++++++++++++++++++++++++++++++++++++++++++++++++
ゼロワンチャンプ
ゼロワンチャンプ | 競走馬データ - netkeiba.com
　牡　鹿毛
{'course_aptitude': 58, 'direct_aptitude': 58, 'race_type': 58, 'growth': 58, 'omobaba': 58, 'status': '', 'sex': '牡', 'color': '鹿毛', 'birthday': '2010/2/3', 'trainer': '森下淳平', 'owner': '国田正忠', 'breeder': 'タイヘイ牧場', 'b

++++++++++++++++++++++++++++++++++++++++++++++++++
ゼンノグッドウッド
ゼンノグッドウッド | 競走馬データ - netkeiba.com
抹消　牡　鹿毛
{'course_aptitude': 73, 'direct_aptitude': 49, 'race_type': 29, 'growth': 38, 'omobaba': 67, 'status': '抹消', 'sex': '牡', 'color': '鹿毛', 'birthday': '2003/3/23', 'trainer': '浅見秀一', 'owner': '大迫久美子', 'breeder': '社台ファーム', 'breed_center': None, 'producefrom': '千歳市', 'seri': None, 'total_prize_chuo': 9247.0, 'total_prize_chiho': 37.0, 'total_race': 41, 'total_win': 6, 'total_result': '6-2-4-29', 'father': 'エアジハード', 'pat_grandfa': 'サクラユタカオー', 'pat_grandmo': 'アイシーゴーグル', 'mother': 'エアジハード', 'mat_grandfa': 'サクラユタカオー', 'mat_grandmo': 'メイクアウイッシュ', 'h_name': 'ゼンノグッドウッド'}
++++++++++++++++++++++++++++++++++++++++++++++++++
ゼンノグライム
ゼンノグライム | 競走馬データ - netkeiba.com
抹消　牡　鹿毛
{'course_aptitude': 58, 'direct_aptitude': 58, 'race_type': 58, 'growth': 58, 'omobaba': 58, 'status': '抹消', 'sex': '牡', 'color': '鹿毛', 'birthday': '2009/4/18', 'trainer': '鈴木伸尋', 'owner': '大迫久美子', 'breeder': '村上欽哉', 'breed_center':

++++++++++++++++++++++++++++++++++++++++++++++++++
ゼンノソルジャー
ゼンノソルジャー | 競走馬データ - netkeiba.com
　牡　栗毛
{'course_aptitude': 96, 'direct_aptitude': 99, 'race_type': 96, 'growth': 66, 'omobaba': 82, 'status': '', 'sex': '牡', 'color': '栗毛', 'birthday': '2003/3/18', 'trainer': '角田輝也', 'owner': '小菅誠', 'breeder': '川島牧場', 'breed_center': None, 'producefrom': '新冠町', 'seri': None, 'total_prize_chuo': 2436.0, 'total_prize_chiho': None, 'total_race': 26, 'total_win': 1, 'total_result': '1-4-1-20', 'father': 'タイキシャトル', 'pat_grandfa': "Devil's Bag", 'pat_grandmo': 'ウェルシュマフィン', 'mother': 'タイキシャトル', 'mat_grandfa': "Devil's Bag", 'mat_grandmo': 'テンジンバトー', 'h_name': 'ゼンノソルジャー'}
++++++++++++++++++++++++++++++++++++++++++++++++++
ゼンノタヂカラオ
ゼンノタヂカラオ | 競走馬データ - netkeiba.com
現役　セ5歳　黒鹿毛
{'course_aptitude': 116, 'direct_aptitude': 38, 'race_type': 38, 'growth': 77, 'omobaba': 96, 'status': '現役', 'sex': 'セ5歳', 'color': '黒鹿毛', 'birthday': '2013/1/6', 'trainer': '田中博康', 'owner': '大迫久美子', 'breeder': '村上欽哉', 'breed_cent

In [48]:
insert_table.insert_many(horse_info_list, chunk_size=5000)

In [40]:
horse_info_list = []

horse_name = 'グッドタルト'

driver.find_element_by_class_name("Txt_Form").send_keys(horse_name)
driver.find_element_by_class_name("Submit_Btn").submit()

wait = WebDriverWait(driver, 20)
# タイトルタグに任意のテキストが含まれたタイトルが表示されるまで20秒待つ
wait.until(EC.title_contains("馬"))
driver.implicitly_wait(10)

print(driver.title)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
if "競走馬検索結果" in driver.title:
    horse_page_info = soup.find_all("td", class_="xml txt_l")
    horse_page_info = horse_page_info + soup.find_all("td", class_="bml txt_l")
    print(horse_page_info)
    horse_page_info = [td.find("a", title=horse_name) for td in horse_page_info if td.find("a", title=horse_name) != None]
#         horse_page_info = soup.find_all("a", title=horse_name)
    print(horse_page_info)
    print("-"*30)
    for horse_info in horse_page_info:
        url = url_fmt + horse_info.get("href")
        html = urlopen(url)
        soup = BeautifulSoup(html, "html.parser")

#         print(soup.find("p", class_="txt_01").text)

        horse_data = create_horsedatadict(soup)
        horse_data['h_name'] = horse_name
        horse_info_list.append(horse_data)
        print(horse_data)
        time.sleep(2)
        print("+"*30)

else:
    try:
        print(soup.find("p", class_="txt_01").text)
        horse_data = create_horsedatadict(soup)
        horse_data['h_name'] = horse_name
        horse_info_list.append(horse_data)
        print(horse_data)
        time.sleep(2)
        print("+"*30)
    except:

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

グッドタルト | 競走馬データ - netkeiba.com
現役　牝7歳　栗毛
{'course_aptitude': 58, 'direct_aptitude': 58, 'race_type': 58, 'growth': 58, 'omobaba': 58, 'status': '現役', 'sex': '牝7歳', 'color': '栗毛', 'birthday': '2011/4/25', 'trainer': '宮徹', 'owner': '杉立恭平', 'breeder': '社台ファーム', 'breed_center': None, 'producefrom': '千歳市', 'seri': '630万円 (2012年 セレクトセール)', 'total_prize_chuo': None, 'total_prize_chiho': None, 'total_race': 4, 'total_win': 0, 'total_result': '0-0-0-4', 'father': 'デュランダル', 'pat_grandfa': 'サンデーサイレンス', 'pat_grandmo': 'サワヤカプリンセス', 'mother': 'デュランダル', 'mat_grandfa': 'サンデーサイレンス', 'mat_grandmo': 'ライラアラウィ', 'h_name': 'グッドタルト'}
++++++++++++++++++++++++++++++
